In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [30]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [6]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [7]:
df_2 = pd.get_dummies(df, columns=["home_ownership","verification_status", "pymnt_plan", "hardship_flag", "debt_settlement_flag", "initial_list_status", 'application_type'])

df_2.head()


,loan_amnt,int_rate,installment,annual_inc,issue_d,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,Mar-2019,low_risk,27.24,0.0,0.0,8.0,...,0,1,0,1,1,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,Mar-2019,low_risk,20.23,0.0,0.0,17.0,...,0,0,1,1,1,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,Mar-2019,low_risk,24.26,0.0,0.0,8.0,...,0,0,1,1,1,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,Mar-2019,low_risk,31.44,0.0,1.0,10.0,...,0,0,1,1,1,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,Mar-2019,low_risk,18.76,0.0,1.0,14.0,...,1,0,0,1,1,1,0,1,1,0


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_3 = df_2.copy()
df_3['next_pymnt_d'] = le.fit_transform(df_3['next_pymnt_d'])
df_3['issue_d'] = le.fit_transform(df_3['issue_d'])
df_3.head()

,loan_amnt,int_rate,installment,annual_inc,issue_d,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,2,low_risk,27.24,0.0,0.0,8.0,...,0,1,0,1,1,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,2,low_risk,20.23,0.0,0.0,17.0,...,0,0,1,1,1,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,2,low_risk,24.26,0.0,0.0,8.0,...,0,0,1,1,1,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,2,low_risk,31.44,0.0,1.0,10.0,...,0,0,1,1,1,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,2,low_risk,18.76,0.0,1.0,14.0,...,1,0,0,1,1,1,0,1,1,0


# Split the Data into Training and Testing

In [9]:
# Create our features
X = df_3.copy()
X = X.drop("loan_status", axis=1)

# Create our target
y = df_3['loan_status']

In [10]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,0.805542,21.778153,0.217766,0.497697,12.587340,0.126030,...,0.478007,0.373992,0.148001,1.0,1.0,1.0,0.123879,0.876121,0.860340,0.139660
std,10277.348590,0.048130,288.062432,1.155800e+05,0.714932,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.499520,0.483865,0.355104,0.0,0.0,0.0,0.329446,0.329446,0.346637,0.346637
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,0.000000,13.890000,0.000000,0.000000,8.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,1.000000,1.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,1.000000,19.760000,0.000000,0.000000,11.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,1.000000,1.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,1.000000,26.660000,0.000000,1.000000,16.000000,0.000000,...,1.000000,1.000000,0.000000,1.0,1.0,1.0,0.000000,1.000000,1.000000,0.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,2.000000,999.000000,18.000000,5.000000,72.000000,4.000000,...,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000


In [12]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [16]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [22]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=1) 
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)

In [27]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.6442107313272958

In [28]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,26,64
Actual 1,8,17107


In [31]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

   high_risk       0.76      0.29      0.42        90
    low_risk       1.00      1.00      1.00     17115

    accuracy                           1.00     17205
   macro avg       0.88      0.64      0.71     17205
weighted avg       1.00      1.00      0.99     17205



In [33]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.08083503998738138, 'total_rec_prncp'),
 (0.0736861378298005, 'total_rec_int'),
 (0.06745906396381002, 'last_pymnt_amnt'),
 (0.06631815305631308, 'total_pymnt_inv'),
 (0.057344783292643, 'total_pymnt'),
 (0.018634317659839493, 'mo_sin_old_rev_tl_op'),
 (0.01715216322372849, 'out_prncp'),
 (0.017077652206699063, 'dti'),
 (0.016796343450732366, 'out_prncp_inv'),
 (0.01597319793708237, 'installment'),
 (0.01562204266410386, 'bc_open_to_buy'),
 (0.015362382442323207, 'issue_d'),
 (0.014990450976737743, 'mo_sin_old_il_acct'),
 (0.014600743860417131, 'max_bal_bc'),
 (0.014371264109953477, 'annual_inc'),
 (0.0142068707520021, 'bc_util'),
 (0.014014630687276056, 'tot_cur_bal'),
 (0.01389221840899042, 'revol_bal'),
 (0.013686871497646176, 'tot_hi_cred_lim'),
 (0.013580724948337768, 'total_rec_late_fee'),
 (0.013471780815655999, 'total_rev_hi_lim'),
 (0.013463786782761385, 'total_bal_il'),
 (0.013454857463554366, 'avg_cur_bal'),
 (0.013068342257530301, 'total_bal_ex_mort'),
 (0.01282536544988

### Easy Ensemble AdaBoost Classifier

In [37]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier 
eec_model = EasyEnsembleClassifier(n_estimators=500, random_state=1) 
eec_model = eec_model.fit(X_train_scaled, y_train)
predictions = eec_model.predict(X_test_scaled)

In [38]:
# Calculated the balanced accuracy score
eec_acc_score = balanced_accuracy_score(y_test, predictions)
eec_acc_score

0.8568068945369559

In [39]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,69,21
Actual 1,908,16207


In [40]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

   high_risk       0.07      0.77      0.13        90
    low_risk       1.00      0.95      0.97     17115

    accuracy                           0.95     17205
   macro avg       0.53      0.86      0.55     17205
weighted avg       0.99      0.95      0.97     17205

